# Analisis de Similitud de SOPs

#### Importo las librerias 

In [20]:
import numpy as np
import pandas as pd
import os
import smart_open
import gensim
import matplotlib.pyplot as plt
import plotly.express as px
from gensim.models import Doc2Vec
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE


#### Paso los documentos de pdf y docx a txt

In [21]:
# %run 1_transformar_formato.py

## Doc2Vec por documento

#### Generacion del Corpus

En read_corpus tokenizamos y etiquetamos los documentos

In [22]:
def read_corpus(fname, tokens_only=False):
    for document in os.listdir(fname):
        file_path = os.path.join(fname, document)
        with smart_open.open(file_path, encoding="iso-8859-1") as f:
            content = f.read()
            tokens = gensim.utils.simple_preprocess(content)
            if tokens_only:
                yield tokens    
            else:
                yield gensim.models.doc2vec.TaggedDocument(tokens, [document])



corpus = list(read_corpus("../txt"))

print(corpus [:2])
print("Documentos leidos: ", len(corpus))

[TaggedDocument(words=['objetivo', 'auditorã', 'as', 'de', 'buenas', 'prã', 'cticas', 'de', 'manufactura', 'gmp', 'buenas', 'prã', 'cticas', 'de', 'distribuciã³n', 'gdp', 'son', 'realizadas', 'frecuentemente', 'para', 'verificar', 'que', 'la', 'fabricaciã³n', 'asã', 'como', 'tambiã', 'el', 'almacenaje', 'distribuciã³n', 'de', 'materiales', 'productos', 'siguen', 'las', 'leyes', 'estã', 'ndares', 'las', 'aplicaciones', 'de', 'autorizaciã³n', 'del', 'mercado', 'las', 'regulaciones', 'de', 'bayer', 'este', 'documento', 'provee', 'los', 'requerimientos', 'mã', 'nimos', 'para', 'la', 'planificaciã³n', 'ejecuciã³n', 'documentaciã³n', 'de', 'las', 'auditorã', 'as', 'gmp', 'gdp', 'iso', 'la', 'auditorã', 'de', 'calidad', 'es', 'uno', 'de', 'los', 'elementos', 'clave', 'del', 'sistema', 'de', 'gestiã³n', 'de', 'cumplimiento', 'de', 'bayer', 'mediante', 'dichas', 'auditorã', 'as', 'se', 'determina', 'el', 'grado', 'de', 'cumplimiento', 'de', 'un', 'proveedor', 'fabricante', 'con', 'los', 'requer

Divido el corpus en un set de entranamiento y uno de testeo

In [23]:
train_corpus, test_corpus = train_test_split(corpus, test_size=0.2, random_state=42)
# print(len(train_corpus))

Guardo el corpus de entranamiento en un txt

In [24]:
with open("corpus.txt", 'w') as file:
    file.write(str(corpus))

#### Entreno el modelo

In [25]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)
model.build_vocab(corpus)
model.train(corpus, total_examples=model.corpus_count, epochs=model.epochs)

In [26]:
print(f"La palabra 'objetivo' aparece {model.wv.get_vecattr('objetivo', 'count')} veces en el corpus de entrenamiento.")

La palabra 'objetivo' aparece 1012 veces en el corpus de entrenamiento.


Genero los archivos de vectores y tensores para poder verlo de forma grafica en [TensorFlow](https://projector.tensorflow.org/)

In [27]:
model.save_word2vec_format('doc_tensor.w2v', doctag_vec=True, word_vec=False)
model.save('doc_tensor_model')
%run 00_process_d2v_files.py -i doc_tensor.w2v


with open("auxFolder/metadata.tsv", 'r') as file:
    lineas = file.readlines()

lineas = [linea.replace("*dt_", "", 1) for linea in lineas]

with open("auxFolder/metadata.tsv", 'w') as file:
    file.writelines(lineas)

FileNotFoundError: [Errno 2] No such file or directory: 'Doc2Vec/auxFolder/vectors.tsv'

#### Testeo el modelo

In [28]:
y_true = []
y_pred = []

# print(len(test_corpus))

model = Doc2Vec.load('doc_tensor_model')
i=0
simil_list = []
for doc in corpus:
    vec = model.infer_vector(doc.words)
    simil = model.dv.most_similar([vec])
    # simil_elem = [[doc[1][0]]]
    # simil_elem[i].append(simil)
    # simil_list.append(simil_elem)
    print("\033[92m" + f"  ########################################{doc[1][0]}#########################################" + "\033[0m")
    for similar_doc in simil:
        print(f"  Document Corpus: {similar_doc[0]} | Input Document: {doc[1][0]} | Similarity: {similar_doc[1]:.4f}")

    predicted_label = simil[0][0]
    y_true.append(doc.tags[0])
    y_pred.append(predicted_label)

  ########################################REGS-AR01-SOP-000002.txt#########################################
  Document Corpus: REGS-AR01-SOP-000002.txt | Input Document: REGS-AR01-SOP-000002.txt | Similarity: 0.9983
  Document Corpus: REGS-AR01-SOP-000272.txt | Input Document: REGS-AR01-SOP-000002.txt | Similarity: 0.9314
  Document Corpus: REGS-AR01-SOP-008231.txt | Input Document: REGS-AR01-SOP-000002.txt | Similarity: 0.8573
  Document Corpus: REGS-AR01-SOP-000273.txt | Input Document: REGS-AR01-SOP-000002.txt | Similarity: 0.8221
  Document Corpus: REGS-AR01-SOP-017051.txt | Input Document: REGS-AR01-SOP-000002.txt | Similarity: 0.7621
  Document Corpus: REGS-AR01-SOP-000405.txt | Input Document: REGS-AR01-SOP-000002.txt | Similarity: 0.7541
  Document Corpus: REGS-AR01-SOP-004739.txt | Input Document: REGS-AR01-SOP-000002.txt | Similarity: 0.7532
  Document Corpus: REGS-AR01-SOP-000748.txt | Input Document: REGS-AR01-SOP-000002.txt | Similarity: 0.7495
  Document Corpus: REGS-AR01

#### Graficamos

In [31]:
vectors = pd.read_csv('auxFolder/vectors.tsv', sep='\t', header=None)

metadata = pd.read_csv('auxFolder/metadata.tsv', sep='\t', header=None)

Debemos reducir la dimensionalidad

##### PCA

In [32]:
pca = PCA(n_components=2)

vectors_red = pca.fit_transform(vectors)

In [33]:
x = vectors_red[:, 0]
y = vectors_red[:, 1]

fig = px.scatter(vectors_red, x, y, hover_name=metadata[0], title='Reduccion por PCA')

fig.show()



##### T-SNE

In [34]:
tsne = TSNE(n_components=2, random_state=42)

vectors_red2 = tsne.fit_transform(vectors)

df = pd.DataFrame(vectors_red2)

df['Name'] = metadata[0]

c:\Program Files\Python312\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning:

Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.

  File "c:\Program Files\Python312\Lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


In [35]:
fig = px.scatter(df, x, y, hover_name='Name', title='Reduccion por T-SNE')

fig.show()